In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils
from pyspark.sql import SparkSession
from pyspark.mllib.classification import LabeledPoint
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from time import perf_counter

In [2]:
# when set local[*], all available cores will be used, the only executor is the driver
conf = SparkConf().setMaster("spark://10.99.250.19:7077").setAppName("HIGGS") 
conf.set("spark.executor.memory", "20G")
conf.set("spark.driver.memory", "2G")
conf.set("spark.executor.cores", "8")
conf.set("spark.cores.max", "16")
# conf.set("spark.executor.instances", "2")
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
# conf.set("spark.default.parallelism", "16")
# create a Spark Session instead of a Spark Context
spark = SparkSession.builder \
    .config(conf = conf) \
  .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/05 13:24:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.option("delimiter", ",").option("header", "false").csv('/work/li.baol/data/HIGGS.csv')

In [4]:
data = df.rdd.map(lambda row: LabeledPoint(float(row['_c0']), list(map(lambda x: float(x), row.asDict().values()))[1:]))

In [5]:
data.take(2)

[LabeledPoint(1.0, [0.869293212890625,-0.6350818276405334,0.22569026052951813,0.327470064163208,-0.6899932026863098,0.7542022466659546,-0.24857313930988312,-1.0920639038085938,0.0,1.3749921321868896,-0.6536741852760315,0.9303491115570068,1.1074360609054565,1.138904333114624,-1.5781983137130737,-1.046985387802124,0.0,0.657929539680481,-0.010454569943249226,-0.0457671694457531,3.101961374282837,1.353760004043579,0.9795631170272827,0.978076159954071,0.9200048446655273,0.7216574549674988,0.9887509346008301,0.8766783475875854]),
 LabeledPoint(1.0, [0.9075421094894409,0.3291472792625427,0.3594118654727936,1.4979698657989502,-0.3130095303058624,1.09553062915802,-0.5575249195098877,-1.588229775428772,2.1730761528015137,0.8125811815261841,-0.2136419266462326,1.2710145711898804,2.214872121810913,0.4999939501285553,-1.2614318132400513,0.7321561574935913,0.0,0.39870089292526245,-1.138930082321167,-0.0008191101951524615,0.0,0.3022198975086212,0.8330481648445129,0.9856996536254883,0.9780983924865723

In [6]:
# (trainingData, testData) = data.randomSplit([0.7, 0.3])

In [7]:
# trainingData.count()

In [8]:
start = perf_counter()

model = DecisionTree.trainClassifier(data, numClasses=2, categoricalFeaturesInfo={},
                                    impurity='gini', maxDepth=8)
end = perf_counter()
end-start                   

52.433826436288655

In [ ]:
# Evaluate model on test instances and compute test error
predictions = model.predict(testData.map(lambda x: x.features))
predictions

In [ ]:
predictions.take(15), testData.map(lambda x: x.label).take(15)

In [ ]:
labels = testData.map(lambda lp: lp.label)
labels

In [ ]:
predsAndLabels = testData.map(lambda lp: (model.predict(lp.features), lp.label))
# predsAndLabels.collect()
# labels = labels.distinct()
# labels.collect()

In [ ]:
metrics = BinaryClassificationMetrics(predsAndLabels)

In [ ]:
filter_label = labels.zipWithIndex().filter(lambda x: x[1] < 100).keys()
filter_pred = predictions.zipWithIndex().filter(lambda x: x[1] < 100).keys()
predsAndLabels = filter_label.zip(filter_pred) #predictions.zip(labels)
predsAndLabels.collect()

In [ ]:
filter_label.glom().map(len).collect(), filter_pred.glom().map(len).collect()

# TODO: Allocate more cores, and see if it's possible to directly construct an RDD. Or try a different serializer

In [ ]:
filter_label.zip(filter_label).take(10)

In [ ]:
filter_pred.collect(), filter_label.collect()

In [ ]:
labels.count()

In [ ]:
predictions.count()

In [ ]:
predsAndLabels.count()

In [ ]:
# testMSE = predsAndLabels.map(lambda lp: (lp[1] - lp[0]) * (lp[1] - lp[0])).sum() /\
#     float(test_data.count())
metrics = BinaryClassificationMetrics(predsAndLabels)    
print(f'AUC = {metrics.areaUnderROC}')
